In [1]:
#importing libraries
import spacy
import pandas as pd
import nltk
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier

In [2]:
#loading the spacy pretrained model on English news
nlp = spacy.load('en_core_web_md')

In [3]:
df1 = pd.read_csv('zomato.csv')
df1.shape

(51717, 17)

In [4]:
#print(df1)

In [5]:
df1.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [6]:
col_list = ['name','reviews_list']
df = df1[col_list]
df.to_csv('zomatoclean.csv')
#df.shape

In [7]:
df = pd.read_csv('zomatoclean.csv')

In [8]:
#Just to test
df = df[:100]

In [9]:
name = df['name']
review = df['reviews_list']

In [10]:
#sentiment score generator function
analyser = SentimentIntensityAnalyzer()
def sentiment(s):
    score = analyser.polarity_scores(s)
    listscore = list(score.values())
    return listscore

In [11]:
for i in range(0,review.size):
    s = str(review.loc[i])
    f = sentiment(s)
    df.loc[i, 'neg_score'] = f[0]*100
    #df.loc[i, 'neut_score'] = f[1]*100
    df.loc[i, 'pos_score'] = f[2]*100
    #df.loc[i, 'comp_score'] = f[3]*100
    
df.to_csv('zomatoscore.csv')

In [12]:
df = pd.read_csv('zomatoscore.csv')
neg = df['neg_score']
pos = df['pos_score']

In [13]:
print(df)

    Unnamed: 0  Unnamed: 0.1                        name  \
0            0             0                       Jalsa   
1            1             1              Spice Elephant   
2            2             2             San Churro Cafe   
3            3             3       Addhuri Udupi Bhojana   
4            4             4               Grand Village   
..         ...           ...                         ...   
95          95            95  Vi Ra's Bar and Restaurant   
96          96            96                      Kaggis   
97          97            97        Ayda Persian Kitchen   
98          98            98                Chatar Patar   
99          99            99                  Polar Bear   

                                         reviews_list  neg_score  pos_score  
0   [('Rated 4.0', 'RATED\n  A beautiful place to ...        3.1       25.8  
1   [('Rated 4.0', 'RATED\n  Had been here for din...        4.4       20.0  
2   [('Rated 3.0', "RATED\n  Ambience is not 

In [15]:
#Finding max contributing word


for i in range(0,review.size):
    
    if(neg.loc[i]<pos.loc[i]):
        pword1 = ''
        pword2 = ''
        maxscore1=0.0
        maxscore2=0.0
        s = str(review.loc[i])
        wlist = s.split()
        for x in wlist:
            f = sentiment(x)
            if(maxscore1<=f[2]*100):
                maxscore1=f[2]*100
                pword1 = x
                
            if(maxscore2<=f[2]*100 and maxscore2!=maxscore1):
                maxscore2=f[2]*100
                pword2 = x
                
        
        df.loc[i,'Max_contributing_word'] = pword1
        df.loc[i,'Secondmax_contributing_word'] = pword2
        #df.loc[i,'Max_pos_score'] = maxscore
        
    elif(neg.loc[i]>=pos.loc[i]):
        nword1 = ''
        nword2 = ''
        minscore1=0.0
        minscore2=0.0
        s = str(review.loc[i])
        wlist = s.split()
        for x in wlist:
            f = sentiment(x)
            if(minscore1<=f[0]*100):
                minscore1=f[0]*100
                nword1 = x
            if(minscore2<=f[0]*100 and minscore2!=minscore1):
                minscore2=f[0]*100
                nword2 = x
        df.loc[i,'Max_contributing_word'] = nword1
        df.loc[i,'Secondmax_contributing_word'] = nword2
        #df.loc[i,'Max_neg_score'] = minscore
                

df.to_csv('zomatoword.csv')    

In [16]:
df = pd.read_csv('zomatoword.csv')
print(df)

    Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1                        name  \
0            0             0               0                       Jalsa   
1            1             1               1              Spice Elephant   
2            2             2               2             San Churro Cafe   
3            3             3               3       Addhuri Udupi Bhojana   
4            4             4               4               Grand Village   
..         ...           ...             ...                         ...   
95          95            95              95  Vi Ra's Bar and Restaurant   
96          96            96              96                      Kaggis   
97          97            97              97        Ayda Persian Kitchen   
98          98            98              98                Chatar Patar   
99          99            99              99                  Polar Bear   

                                         reviews_list  neg_score  pos_score  \
0   [('R